In [1]:
import streamlit as st
from core.settings import settings
import pandas as pd
import src.apify_functions as apify

In [2]:
def query_airbnbs(client, run_input):
    actor = client.actor("dtrungtin/airbnb-scraper")
    run = actor.call(run_input=run_input)
    print("--- Run succeeded ---")
    unpackaged = actor.last_run(status='SUCCEEDED')
    unpackaged_json = unpackaged.dataset().list_items().items
    print("--- Results retrieved ---")

    parsed = []

    for item in unpackaged_json:
        values = []

        values.append(item["url"])
        values.append(item["name"])

        try:
            values.append(item["stars"])
        except KeyError:
            values.append("no_stars")

        values.append(item["numberOfGuests"])
        values.append(item["address"])
        values.append(item["roomType"])
        values.append(item["location"])
        values.append(item["pricing"])
        values.append(len(item["reviews"]))

        parsed.append(values)


    df = pd.DataFrame(parsed, columns=["url", "name", "stars", "n_guests", "address", "roomType", "location", "pricing", "reviews"])
    print("--- Results parsed ---")

    return df

In [3]:
client = apify.authenticate_apify(
    token=settings.APIFY_API_KEY
)

In [4]:
run_input = {
    "locationQuery": "Monterrey, ",
    "maxListings": 10,
    "includeReviews": True,
    "maxReviews": 10,
    "startUrls": [],
    "calendarMonths": 0,
    "currency": "MXN",
    "proxyConfiguration": { "useApifyProxy": True },
    "maxConcurrency": 50,
    "limitPoints": 100,
    "timeoutMs": 60000,
}

In [5]:
df = query_airbnbs(client=client, run_input=run_input)

2023-06-11 14:40:31.571 INFO    httpx: HTTP Request: POST https://api.apify.com/v2/acts/dtrungtin~airbnb-scraper/runs "HTTP/1.1 201 Created"
2023-06-11 14:40:32.640 INFO    httpx: HTTP Request: GET https://api.apify.com/v2/actor-runs/D88jy3rvHn8QvDqUQ?waitForFinish=999999 "HTTP/1.1 200 OK"
